In [12]:
train_root = "chest_xray/train"
test_root = "chest_xray/test"


In [13]:
from keras.preprocessing.image import ImageDataGenerator
# nvidia-smi 
batch_size = 20

Generator = ImageDataGenerator()
train_data = Generator.flow_from_directory(
    train_root, (150, 150), batch_size=batch_size)
test_data = Generator.flow_from_directory(
    test_root, (150, 150), batch_size=batch_size,shuffle= False)
import os
import tensorflow as tf
num_classes = len([i for i in os.listdir(train_root)])

Found 5216 images belonging to 2 classes.
Found 624 images belonging to 2 classes.


In [14]:
import keras
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D

model = Sequential()

model.add(Conv2D(32, (3, 3), input_shape=(150, 150, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2), strides=2))
model.add(Dropout(0.3))

model.add(Conv2D(32, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2), strides=2))
model.add(Dropout(0.3))

model.add(Conv2D(64, (3, 3),activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2), strides=2))
model.add(Dropout(0.2))

model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2), strides=2))
model.add(Dropout(0.2))

model.add(Flatten())

model.add(Dense(128, activation='relu'))
model.add(Dropout(0.2))

model.add(Dense(64, activation='relu'))
model.add(Dropout(0.2))

model.add(Dense(num_classes, activation="softmax"))
model.summary()


Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_8 (Conv2D)           (None, 148, 148, 32)      896       
                                                                 
 max_pooling2d_8 (MaxPooling  (None, 74, 74, 32)       0         
 2D)                                                             
                                                                 
 dropout_12 (Dropout)        (None, 74, 74, 32)        0         
                                                                 
 conv2d_9 (Conv2D)           (None, 72, 72, 32)        9248      
                                                                 
 max_pooling2d_9 (MaxPooling  (None, 36, 36, 32)       0         
 2D)                                                             
                                                                 
 dropout_13 (Dropout)        (None, 36, 36, 32)       

In [15]:
# remove optimizer if needed
model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=tf.keras.optimizers.Adam(), metrics=['accuracy'])
model.fit(train_data, batch_size=batch_size, epochs=1)


261/261 [==============================] - 91s 346ms/step - loss: 2.0254 - accuracy: 0.7193


In [16]:
score = model.evaluate(train_data)
print("训练集准确率情况：",score[1])
import numpy as np
pred = model.predict(test_data)
pred=np.argmax(pred,axis=1)
score = model.evaluate(test_data)
print("测试集准确率情况：",score[1])

261/261 [==============================] - 43s 165ms/step - loss: 0.5819 - accuracy: 0.7941
训练集准确率情况： 0.7940950989723206
32/32 [==============================] - 5s 151ms/step - loss: 0.6221 - accuracy: 0.6474
测试集准确率情况： 0.6474359035491943


In [17]:
from keras.models import save_model
save_model(model, "Pneumonia")
#model.save("C:/Users/SaltyFishEx/OneDrive/2-1/Flower/assets", save_format="h5")

INFO:tensorflow:Assets written to: Pneumonia\assets


In [19]:
from keras.models import load_model
from PIL import Image
import numpy as np

model = load_model("Pneumonia")

import cv2
image = cv2.imread("chest_xray/val/NORMAL/NORMAL2-IM-1442-0001.jpeg")
gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
img = cv2.merge([gray,gray,gray])

img.resize((150,150,3))
img = np.asarray(img, dtype="float32") #need to transfer to np to reshape
img = img.reshape(1, img.shape[0], img.shape[1], img.shape[2]) #rgb to reshape to 1,100,100,3
pred=model.predict(img)
print(img.shape , pred)

(1, 150, 150, 3) [[0.44266954 0.5573304 ]]
